# LIST OF CUSTOM OFFICES AT THE FRENCH POINTS OF ENTRY INTO THE EUROPEAN UNION (AIRPORTS)

## *Scraping/Parsing/Cleaning/Reshaping/Translating data from a PDF*

The data will be extracted from a PDF containing [a list of the French points of entry into the European Union and their respective custom offices.](https://www.douane.gouv.fr/sites/default/files/uploads/files/Services-en-ligne/ICS/ics-liste-points-d-entree-francais.pdf)

As stated by the French Customs:

>* **"The names of the airports which constitute the first points of entry, or subsequent points, are included here, as well as the codes of the customs offices which are attached to them."**
* **"All the air entry points open to international traffic, on which flights from third countries are likely to land, are also listed."**

The data related to airports in the file `ics-liste-points-d-entree-francais.pdf` starts on page 3, ends on page 7, and the table on each page has headers.

The data will be extracted using the [`pdfplumber`](https://github.com/jsvine/pdfplumber) tool.

#### Steps:
1. Import dependencies
2. Open the PDF and check its structure
3. Create an empty data frame and specify the columns
4. Create a function to extract data from a single PDF page and return a data frame
5. Loop over the pages and call the function on each page
6. Clean up and translate the data
7. Perform a quick basic analysis in pandas
8. Write the data to a CSV file

### 1. Import dependencies

In [1]:
import pdfplumber
import pandas as pd

In [2]:
# Set pandas display options (optional)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", -1)

### 2. Open the PDF and check its structure

Using `pdfplumber`'s syntax to open a file, explore the PDF structure and extract the airports data table located on page 3.

In [3]:
with pdfplumber.open("ics-liste-points-d-entree-francais.pdf") as pdf:
    
    # print(pdf)
    # print(pdf.pages)
    test = pdf.pages[3]
    table = test.extract_table()
    print(table)

[["Nom du Point d'entrée", 'Nom \ndu bureau', 'Code\nBureau', 'Trafic', 'Code \nIATA', 'Code \nOACI'], ['►Aéroports ayant un trafic de fret pays tiers', None, None, None, None, None], ['Agen La Garenne', 'Agen', 'FR005130', 'Aéroport (AIR)', 'AGF', 'LFBA'], ['Bâle Mulhouse', 'Bâle Mulhouse aéroport', 'FR000340', 'Aéroport (AIR)', 'BSL\nMLH', 'LFBD'], ['Bordeaux Mérignac', 'Bordeaux Mérignac', 'FR000600', 'Aéroport (AIR)', 'BOD', 'LFBD'], ['Brest Guipavas', 'Brest', 'FR000690', 'Aéroport (AIR)', 'BES', 'LFRB'], ['Châteauroux Déols', 'Châteauroux', 'FR000910', 'Aéroport (AIR)', 'CHR', 'LFLX'], ['Deauville Saint-Gatien', 'Caen', 'FR000720', 'Aéroport (AIR)', 'DOL', 'LFRG'], ['Dinard Pleurtuit', 'Saint-Malo', 'FR004060', 'Aéroport (AIR)', 'DNR', 'LFRD'], ['Évreux', 'Rouen port', 'FR003920', 'Aéroport (AIR)', 'DPE', 'LFAB'], ['Lille Lesquin', 'Lesquin', 'FR005350', 'Aéroport (AIR)', 'LIL', 'LFQQ'], ['Lyon Saint-Exupéry', 'Lyon aéroport', 'FR002650', 'Aéroport (AIR)', 'LYS', 'LFLL'], ['Marse

### 3. Create an empty data frame and define the columns

In [4]:
cols = ["point_of_entry_name", "office_name", "office_code", "traffic", "iata_code", "icao_code"]

df = pd.DataFrame(columns=cols)

### 4. Create a function to extract data from a single PDF page

This function will be called on every PDF page we hand it. It will take a `pdfplumber.Page` object, extract the table and return the data in a data frame with the same headers as the empty one we just created.

In [5]:
def page_to_df(page):
    
    # Find the table on the page and extract the data
    table = page.extract_table()
    
    # Grab all rows in the table except for the first one,
    # which is the header row
    lines = table[1:]
    
    # Return the data in a dataframe
    return pd.DataFrame(lines, columns=cols)

### 5. Loop over the pages and call the function on each page

As we extract the data from each page, we'll append the data frame returned by our function to the empty data frame (`df`) that we created earlier.

In [6]:
# Open the PDF
with pdfplumber.open("ics-liste-points-d-entree-francais.pdf") as pdf:
    
    # Skip the first 3 pages that do not contain the data table we need
    pages_with_data = pdf.pages[3:]
    
    # Loop over the pages with data
    for page in pages_with_data:
        
        # Call the extraction function to grab the data from this page
        df_to_append = page_to_df(page)
        
        # Append it to our main data frame, chopping off the index column
        df = df.append(df_to_append, ignore_index=True)

In [7]:
df

,point_of_entry_name,office_name,office_code,traffic,iata_code,icao_code
0,►Aéroports ayant un trafic de fret pays tiers,None,None,None,None,None
1,Agen La Garenne,Agen,FR005130,Aéroport (AIR),AGF,LFBA
2,Bâle Mulhouse,Bâle Mulhouse aéroport,FR000340,Aéroport (AIR),BSL\nMLH,LFBD
3,Bordeaux Mérignac,Bordeaux Mérignac,FR000600,Aéroport (AIR),BOD,LFBD
4,Brest Guipavas,Brest,FR000690,Aéroport (AIR),BES,LFRB
5,Châteauroux Déols,Châteauroux,FR000910,Aéroport (AIR),CHR,LFLX
6,Deauville Saint-Gatien,Caen,FR000720,Aéroport (AIR),DOL,LFRG
7,Dinard Pleurtuit,Saint-Malo,FR004060,Aéroport (AIR),DNR,LFRD
8,Évreux,Rouen port,FR003920,Aéroport (AIR),DPE,LFAB
9,Lille Lesquin,Lesquin,FR005350,Aéroport (AIR),LIL,LFQQ


### 6. Clean up the data

In [8]:
# Kill line breaks
df.replace("\n", " ", inplace=True, regex=True)

In [9]:
# Make all values uppercase and strip whitespace in all columns
df["point_of_entry_name"] = df.point_of_entry_name.str.upper().str.strip()
df["office_name"] = df.office_name.str.upper().str.strip()
df["office_code"] = df.office_code.str.strip()
df["traffic"] = df.traffic.str.upper().str.strip()
df["iata_code"] = df.iata_code.str.strip()
df["icao_code"] = df.icao_code.str.strip()

In [10]:
# Remove all French accents from strings in columns "point_of_entry_name", "office_name" and "traffic"
df["point_of_entry_name"] = df.point_of_entry_name.str.normalize("NFKD") \
                                                  .str.encode("ascii", errors="ignore").str.decode("utf-8")
df["office_name"] = df.office_name.str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
df["traffic"] = df.traffic.str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")

In [11]:
# Select indexes where vector values is equal to "Aéroports ayant un trafic de fret pays tiers"
# Assign "Aéroport ayant un trafic de fret pays tiers" to new column "airport_type"
df.loc[0:30, "airport_type"] = "AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC"

In [12]:
# Select indexes where vector values is equal to "Aéroports habilités à recevoir du fret pays tiers"
# Assign "Aéroport habilités à recevoir du fret pays tiers" value to column "airport_type"
df.loc[30:123, "airport_type"] = "AIRPORT AUTHORIZED TO RECEIVE THIRD COUNTRY FREIGHT"

In [13]:
# Select indexes where territory values is equal to "Metropole"
# Assign entries to new column "territory"
df.loc[0:24, "territory"] = "MAINLAND"
df.loc[30:123, "territory"] = "MAINLAND"

In [14]:
# Select indexes where territory values is equal to "DOM"
# Assign entries to column "territory"
df.loc[24:29, "territory"] = "OVERSEAS"

In [15]:
# Remove all indexes containing vector headers
# Reset index
df = df.drop([0, 13, 30, 46, 81, 116], axis=0).reset_index(drop=True)

In [16]:
# Remove the index containing a territory header
# Reset index
df = df.drop([22], axis=0).reset_index(drop=True)

In [17]:
# Replace unique value "AEROPORT (AIR)" in column "traffic" with its English translation
df["traffic"] = "AIRPORT (AIR)"

### 7. Perform a quick basic analysis in pandas

Make a quick analysis and write _an entire journalism sentence_ reporting the number of airports with third country freight traffic located in Mainland France that are first points of entry into the European Union as well as the percentage of the total number of airports in the data frame it represents.

In [18]:
# What is the total number of airports (how many records are there?)
record_count = len(df)

# Filter for only airports with third country freight traffic in Mainland France
airports = df[(df.airport_type == "AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC") & (df.territory == "MAINLAND")]

# How many of those are there?
airports_count = len(airports)

# Calculate the percentage of the whole
pct_whole = (airports_count / record_count) * 100

Formulate a journalistic sentence.

In [19]:
# Write out a formatted sentence using an f-string
story_sentence = f'Of the {record_count:,} French airports which constitute the first points of entry into the European Union, {airports_count:,} ({pct_whole:0.2f}%) are airports with third country freight traffic that are located in Mainland France.'

print(story_sentence)

Of the 116 French airports which constitute the first points of entry into the European Union, 22 (18.97%) are airports with third country freight traffic that are located in Mainland France.


### 8. Write the data to a CSV file

In [20]:
df.to_csv("list-of-french-points-of-entry-in-eu-airports.csv", sep=",", encoding="utf-8", index=False)

In [21]:
data = pd.read_csv("list-of-french-points-of-entry-in-eu-airports.csv")

data

,point_of_entry_name,office_name,office_code,traffic,iata_code,icao_code,airport_type,territory
0,AGEN LA GARENNE,AGEN,FR005130,AIRPORT (AIR),AGF,LFBA,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
1,BALE MULHOUSE,BALE MULHOUSE AEROPORT,FR000340,AIRPORT (AIR),BSL MLH,LFBD,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
2,BORDEAUX MERIGNAC,BORDEAUX MERIGNAC,FR000600,AIRPORT (AIR),BOD,LFBD,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
3,BREST GUIPAVAS,BREST,FR000690,AIRPORT (AIR),BES,LFRB,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
4,CHATEAUROUX DEOLS,CHATEAUROUX,FR000910,AIRPORT (AIR),CHR,LFLX,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
5,DEAUVILLE SAINT-GATIEN,CAEN,FR000720,AIRPORT (AIR),DOL,LFRG,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
6,DINARD PLEURTUIT,SAINT-MALO,FR004060,AIRPORT (AIR),DNR,LFRD,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
7,EVREUX,ROUEN PORT,FR003920,AIRPORT (AIR),DPE,LFAB,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
8,LILLE LESQUIN,LESQUIN,FR005350,AIRPORT (AIR),LIL,LFQQ,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
9,LYON SAINT-EXUPERY,LYON AEROPORT,FR002650,AIRPORT (AIR),LYS,LFLL,AIRPORT WITH THIRD COUNTRY FREIGHT TRAFFIC,MAINLAND
